In [1]:
import pandas as pd
import numpy as np

# 1.读取数据

In [2]:
df = pd.read_excel("./bin_data.xlsx")
df.head()

,年龄,收入,孩子数量,是否违约
0,46,0,0,0
1,34,3200,4,1
2,31,3300,3,1
3,39,1500,0,1
4,32,0,3,0


# 2.分割数据集

In [16]:
def split_data(dataframe,y_name,test_size):
    from sklearn.model_selection import train_test_split
    y = dataframe[y_name]
    x = dataframe.drop(y_name,axis = 1)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=test_size,random_state = 0)
    return x_train,x_test,y_train,y_test

In [17]:
x_train,x_test,y_train,y_test = split_data(dataframe = df,y_name="是否违约",test_size=0.3)

# 3.构建决策树

In [31]:
from sklearn.tree import DecisionTreeClassifier
dtc1 = DecisionTreeClassifier()
dtc1.fit(np.array(x_train["年龄"]).reshape((-1,1)),y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
dtc2 = DecisionTreeClassifier()
dtc2.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [112]:
# dtc1.predict(np.array(x_train["年龄"]).reshape((-1,1)))
# dtc1.predict(np.array(x_test["年龄"]).reshape((-1,1)))

## 3.1决策树estimator的参数
1. criterion: 子节点分裂用哪一种衡量标准，有“gini"-->基尼系数；”entropy"--->信息增益
2. splitter: 每个节点处选择拆分的策略，有根据criterion得出的“best"或者随机拆分”random"---->一般不使用这个参数
3. max_depth:树的最大深度。int/None,默认未进行设置---》**常用**
4. min_samples_split:子节点分裂需要的最小样本数，默认等于2
5. min_samples_leaf:叶子节点最少样本数,默认等于1---》**常用**
6. min_weight_fraction_leaf:
7. max_features:类似于随机森林的最大特征数量，默认等于总特征数-----》一般不使用这个参数
8. random_state:默认等于None.比较每个特征时，特征排列顺序是随机的---》**必须确定**
9. max_leaf_nodes:最大叶子节点数量
10. min_inpurity_descrease:最小不纯度减少的值.如果分裂后的不纯度减少程度大于等于当前不纯度减少程度，则分裂。默认等于0，也就是有减少就分裂
11. min_inpurity_split:当前子节点的不纯度大于阈值，就继续分裂。默认=1e-7
12. class_weight:默认None，可选“balanced"或自己给定
> 1. 如果是二分类，如果不给，那么默认类别权重1:1
> 2. 如果是多分类，那么不是写成 [{1:1}, {2:5}, {3:1}, {4:1}].而是多分类转成二分类， [{0: 1, 1: 1}, {0: 1, 1: 5}, {0: 1, 1: 1}, {0: 1, 1: 1}] 
> 3. ”balanced"表示根据不同类别样本数量自动调整权重
13. presort:是否对数据进行预排序，加快fit时最佳分割点的寻找。默认为False，改为True可提速

## 3.2 决策树fit好的模型拥有的属性
1. classes_: 模型预测的类的标签
2. feature_importances_:对应列名给出每个特征的重要性值
3. max_features_:fit模型用到的最大特征数，默认等于总特征数
4. n_classes:预测的类别个数
5. n_features:fit模型的总特征数
6. tree_:树对象
> 1. node_count:决策树的节点计数，内部+叶节点---等同于capacity
> 2. 

## 3.3 决策树模型拥有的方法
1.  estimator.apply(x_test)---返回每个样本被预测所在的叶子节点的节点索引
2.  estimator.decision_path(x_test)---返回在决策树中的决策路径
3.  estimator.fit()拟合数据
4.  estimator.get_params()---返回决策树内部参数
5.  estimator.predict()----预测数据集
6.  estimator.predict_proba()----预测样本的概率
7.  estimator.score()返回评价准确率
8. estimator.set_params()----设置决策树内部参数

In [98]:
dtc1.set_params(max_depth = 10)
dtc1.fit(np.array(x_train["年龄"]).reshape((-1,1)),y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

# 4.交叉验证

In [100]:
def cross_val(estimator,X,y,k,scoring):
    """进行交叉验证，得到交叉验证的分数"""
    from sklearn.model_selection import cross_val_score
    scores = cross_val_score(estimator=estimator,X=X,y=y,cv = k,scoring=scoring)
    scores_mean = scores.mean()
    return scores_mean

# 5.网格搜索
1. 设置参数
> 1. estimator: 估计器对象，必须含有score函数
> 2. param_grid：参数网格，以字典呈现要搜索的参数。key为参数名，value为该参数搜索的范围的列表
> 3. scoring:结果评价标准
> 4. cv:交叉验证次数
> 5. n_jobs:并行数，默认为1，,1表示与cpu核数一致
2. 网格搜索的属性方法
> 1. grid.fit(train_x,trian_y)--->运行网格搜索
> 2. grid.grid_scores_:给出所有参数下每种组合的分数
> 3. grid.best_score_:给出最优评分
> 4. grid.best_params_:给出最优评分下的参数组合

In [3]:
def transform_data(data,X,y):
    """1. 将单列数据转换成estimator需要的格式"""
    X = np.array(data[X]).reshape((-1,1))
    y = data[y]
    return X,y

In [4]:
X,y = transform_data(data = df,X ="年龄",y = "是否违约")

In [5]:
def grid_search(estimator,param_grid,X,y,scoring = "roc_auc"):
    """2. 用于决策树分箱的决策树训练网格搜索，返回最佳超参数"""
    from sklearn.model_selection import GridSearchCV
    grid = GridSearchCV(estimator=estimator,param_grid= param_grid,scoring = scoring,cv=3,n_jobs = -1)
    grid.fit(X=X,y=y)
    print("网格搜索得到的最佳%s为%s" %(scoring,grid.best_score_))
    dic = grid.best_params_
    return dic

In [7]:
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier()
param_grid = {"max_depth":range(1,8),"min_samples_leaf":range(1,100)}
scoring = "roc_auc"
dic = grid_search(estimator,param_grid,scoring = scoring,X=X,y=y)
dic

网格搜索得到的最佳roc_auc为0.6582526666666667


{'max_depth': 3, 'min_samples_leaf': 97}

In [8]:
dtc = DecisionTreeClassifier(max_depth=dic.get("max_depth"),min_samples_leaf = dic.get("min_samples_leaf"))
dtc.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=97, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
threshold = dtc.tree_.threshold
splits = threshold[threshold > 0]
split = np.sort(splits)
split

array([21.5, 22.5, 23.5, 27.5, 35.5, 45.5, 57.5])

In [148]:
pd.cut(ori_data,bins,right=True,labels=None,include_lowest=False,retbins=False)

array([[46],
       [34],
       [31],
       ...,
       [32],
       [31],
       [27]])

In [11]:
labels = []
for i in range(len(split[:-1])):
    t = str(split[i]) + "-" + str(split[i+1])
    labels.append(t)
labels

['21.5-22.5', '22.5-23.5', '23.5-27.5', '27.5-35.5', '35.5-45.5', '45.5-57.5']

In [ ]:
df.loc

In [25]:
bin0 = "<="+str(split[0])
bin_last = ">" + str(split[-1])
index0 = (df["年龄"] <= split[0])
index_last = df["年龄"] > split[-1]
df["年龄_bin"] = pd.cut(df["年龄"],bins = split,labels=labels)
index0
# df.loc[index_last,"年龄"] = bin_last
# df["年龄_bin"] = df["年龄_bin"].astype("category")
# df.loc[df["年龄_bin"].isnull(),"年龄_bin"] = "新类别"
# df.loc[index_last,"年龄_bin"] = "000"
# df.head()

TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'str'

In [158]:
pd.cut(df["年龄"],bins = split,labels=labels)

0       45.5-57.5
1       27.5-35.5
2       27.5-35.5
3       35.5-45.5
4       27.5-35.5
5       22.5-23.5
6       35.5-45.5
7       27.5-35.5
8       23.5-27.5
9       23.5-27.5
10            NaN
11      35.5-45.5
12      22.5-23.5
13      35.5-45.5
14      35.5-45.5
15      23.5-27.5
16      45.5-57.5
17      27.5-35.5
18      35.5-45.5
19      35.5-45.5
20      27.5-35.5
21            NaN
22      23.5-27.5
23      23.5-27.5
24      45.5-57.5
25      35.5-45.5
26      27.5-35.5
27      27.5-35.5
28      27.5-35.5
29      23.5-27.5
          ...    
2970    35.5-45.5
2971    35.5-45.5
2972    23.5-27.5
2973    27.5-35.5
2974          NaN
2975    27.5-35.5
2976    23.5-27.5
2977    23.5-27.5
2978    27.5-35.5
2979    21.5-22.5
2980    35.5-45.5
2981    27.5-35.5
2982    23.5-27.5
2983    35.5-45.5
2984          NaN
2985    27.5-35.5
2986    45.5-57.5
2987    35.5-45.5
2988    35.5-45.5
2989    27.5-35.5
2990    27.5-35.5
2991          NaN
2992    45.5-57.5
2993    27.5-35.5
2994    27

In [ ]:
def tree_bin(data,x,y,dispersed = True):
    from sklearn.model_selection import train_test_split
    y = data[y]
    x = 
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 0)
    
    
    if dispersed:
        from sklearn.tree import DecisionTreeClassifier
        